In [13]:
import requests
import pandas as pd
import base64
import json

In [130]:
import os
import json

def get_longest_category(categories):
    # Seleccionar la categoría más larga
    longest_category = max(categories, key=len) if categories else ''
    # Dividir la categoría en niveles individuales
    return longest_category.strip('/').split('/')

def process_json_file(filepath):
    """Procesa un archivo JSON dado su path."""
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data # Ejemplo de procesamiento

def iterate_json_files_in_directory(directory_path):
    all_products = []
    """Itera sobre archivos JSON en el directorio dado."""
    for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            data = process_json_file(file_path)
            products =  data.get('data',{}).get('search',{}).get('products',[]).get('edges',[])
            for product in products:
                # Extrae la información deseada
                node = product.get('node',{})
                product_name = node.get('name','No disponible')
                link = node.get('slug', 'No disponible')
                try:
                    selling_price = node.get('sellers','')[0].get('commertialOffer','').get('Price','No disponible')
                    list_price = node.get('sellers','')[0].get('commertialOffer','').get('PriceWithoutDiscount','No disponible')
                except:
                    selling_price = 'No disponible'
                    list_price = 'No disponible'
                categories = node.get('categories', [])
                longest_category_levels = node.get('breadcrumbList',[]).get('itemListElement',[])[-2].get('item','').strip('/').split('/')
    
                all_products.append({
                    **{f'Category Level {i+1}': longest_category_levels[i] if i < len(longest_category_levels) else None for i in range(len(longest_category_levels))},
                    'Nombre': product_name,
                    'Link': 'https://www.exito.com/'+ link + '/p' ,
                    'Precio promocional': selling_price,
                    'Precio regular': list_price,
            
                })
    df = pd.DataFrame(all_products)
    df.rename(columns = {
    'Category Level 1':'Divisón',
    'Category Level 2':'Categoría',
    'Category Level 3':'Subcategoría',
    }, inplace=True)
    df = df.drop(df.columns[-1], axis=1)
    return df
    
    
# Cambia 'ruta/a/tu/carpeta' por el path a tu carpeta con archivos JSON
directory_path = 'C:\\Users\\mxrojas\\PycharmProjects\\JMC - SCRAPING\\MercaWeek_2024\\json_exito'

In [131]:
df = iterate_json_files_in_directory(directory_path)

In [132]:
df.head()

,Divisón,Categoría,Subcategoría,Nombre,Link,Precio promocional,Precio regular
0,mercado,lacteos-huevos-y-refrigerados,leche,Leche Deslactosada ALPINA Bolsa x6und 1100ml (...,https://www.exito.com/leche-deslactosada-sixpa...,31940,31940
1,mercado,lacteos-huevos-y-refrigerados,quesos-quesitos-y-cuajadas,Queso Mozarella FINESSE 30 Tajadas(450 gr),https://www.exito.com/queso-fresco-15-pcto-de-...,20512,27350
2,mercado,lacteos-huevos-y-refrigerados,yogurt-y-bebidas-lacteas,Alimento Lácteo BON YURT Zucaritas x4und 170gr...,https://www.exito.com/alimento-lacteo-con-cere...,9210,15350
3,mercado,lacteos-huevos-y-refrigerados,yogurt-y-bebidas-lacteas,Yogurt ALPINA Original multisabor x5und vaso (...,https://www.exito.com/yogurt-pack-x5-unidades-...,9450,12600
4,mercado,lacteos-huevos-y-refrigerados,yogurt-y-bebidas-lacteas,Alim Lacteo Ferm X8 Unds Bolsa YOGO YOGO 1200 gr,https://www.exito.com/yogures-en-bolsa-paq-x-8...,8137,10850


In [133]:
df.to_excel('MercaWeek_Exito.xlsx', index=False)